In [1]:
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt

import glob
import random
import tensorflow as tf
from shutil import rmtree
from keras.models import load_model
from os.path import join, getctime, basename
from models import *
from utils import *
from data_preprocess import load_prep_data

%load_ext autoreload
%autoreload 2

aug_data = load_prep_data(time=500, debug=True, onehot=True)
x_train = aug_data['x_train']
y_train = aug_data['y_train']
x_valid = aug_data['x_valid']
y_valid = aug_data['y_valid']
x_test = aug_data['x_test']
y_test = aug_data['y_test']

Shape of x_train set: (7191, 22, 250)
Shape of y_train labels: (7191, 4)
Shape of x_valid set: (1269, 22, 250)
Shape of y_valid labels: (1269, 4)
Shape of x_test set: (1772, 22, 250)
Shape of y_test labels: (1772, 4)
Shape of x_train_valid set: (8460, 22, 250)
Shape of y_train_valid labels: (8460, 4)


In [2]:
input_shape = x_train.shape
x_train = tf.transpose( tf.expand_dims(x_train, axis=-1), perm=[0, 2, 3, 1])
x_valid = tf.transpose( tf.expand_dims(x_valid, axis=-1), perm=[0, 2, 3, 1])

print("x_train.shape: {}", format(x_train.shape))

x_train.shape: {} (7191, 250, 1, 22)


2024-03-15 21:03:33.756961: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-15 21:03:33.756994: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-15 21:03:33.757006: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-15 21:03:33.757024: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-15 21:03:33.757046: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
config = {
    # Network
    'num_inputs': x_train.shape[0],
    'input_shape': (input_shape[2],1, input_shape[1]),
    'epochs': 50,
    'batch_size': 128, 
    'lr': 0.0001
}
SpatialConvNet1 = SpatialConvNet()
SpatialConvNet1.build_model(config)
history = SpatialConvNet1.train(x_train, y_train, x_valid, y_valid, config, get_workpath('SpatialConvNet'))

/Users/peiyuanlee/miniforge3/envs/myenv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 250, 1, 128)    │        31,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 250, 1, 128)    │       196,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 250, 1, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_2             │ (None, 250, 1, 128)    │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 250, 1, 64)     │       196,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 250, 1, 64)     │        98,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 250, 1, 64)     │        98,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 250, 1, 64)     │        98,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 250, 1, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 13, 1, 64)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_3             │ (None, 13, 1, 64)      │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 13, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 13, 152)        │        85,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 13, 152)        │           608 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1976)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         7,908 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 814,628 (3.11 MB)

 Trainable params: 813,940 (3.10 MB)

 Non-trainable params: 688 (2.69 KB)

Model compiled.
Epoch 1/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.2684 - loss: 8.2336 - val_accuracy: 0.3625 - val_loss: 7.2689
Epoch 2/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.3139 - loss: 7.4306 - val_accuracy: 0.4460 - val_loss: 6.7421
Epoch 3/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.3704 - loss: 6.8809 - val_accuracy: 0.5083 - val_loss: 6.3031
Epoch 4/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 52s 911ms/step - accuracy: 0.4013 - loss: 6.4714 - val_accuracy: 0.5201 - val_loss: 5.9638
Epoch 5/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 56s 991ms/step - accuracy: 0.4327 - loss: 6.1308 - val_accuracy: 0.5626 - val_loss: 5.6748
Epoch 6/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 53s 925ms/step - accuracy: 0.4584 - loss: 5.8408 - val_accuracy: 0.6107 - val_loss: 5.4105
Epoch 7/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 53s 936ms/step - accuracy: 0.4695 - loss: 5.6307 - val_accuracy: 0.5957 - val_loss: 5.2350
Epoch 8/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 52s 918ms/step - accuracy: 0.4984 - loss: 5.3900 - v

In [9]:
x_test_transposed = tf.transpose(tf.expand_dims(x_test, axis=-1), perm=[0, 2, 3, 1])
raw = SpatialConvNet1.evaluate(x_test_transposed, y_test)
print("Raw Acc result: {}".format(raw[1]))

56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 157ms/step - accuracy: 0.7049 - loss: 1.9948
Raw Acc result: 0.7116252779960632
